In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from shapely.geometry import Point
import matplotlib.pyplot as plt
from rasterio.mask import mask
from rasterio.plot import show
from rasterio.plot import plotting_extent
from natsort import natsorted
import gdal
import pandas as pd
import numpy as np
import geopandas
import rasterio
import pycrs
import math

file_dir=r'C:/Users/Mark.Rademaker/PycharmProjects/InternshipNaturalis/trait-geo-diverse-dl/data_GIS_extended'

In [ ]:
#access file with list of taxa names
taxa=pd.read_csv(file_dir+"/data/SQL_filtered_gbif/taxa_list.txt",header=None)
taxa.columns=["taxon"]

species_occ_dict={}

for i in taxa["taxon"]:
    taxon_data = pd.read_csv(file_dir+"/data/SQL_filtered_gbif/%s_filtered_data.csv"%i)
    #add species dataframe to dict
    species_occ_dict["%s"%i] = taxon_data  
    #check whether all species have been included and inspect dictionary
if len(species_occ_dict.keys())==len(taxa["taxon"]):
    print("All species dataframes now in dictionary")
else:
    print("Error: not all species dataframe included")

#### Part 1
- First read in occurrence data
- Create a copy that we can use in the original state later
- Create a buffer around each occurrence point, merge it into a single polygon
- Clip the environmental raster based on this extend

In [ ]:
for key in species_occ_dict:  
    #load occurrence data and set initial projection
    data=species_occ_dict[key]
    #print(data.columns)
    spec = key
    print(spec)

    data['coordinates'] = list(zip(data["decimal_longitude"], data["decimal_latitude"]))
    data['coordinates'] = data["coordinates"].apply(Point)
    data["present/pseudo_absent"]=1
    geo_data=geopandas.GeoDataFrame(data, geometry='coordinates',crs={'init' :'epsg:4326'})

    #change projection to azimuthal equidistant to calculate 1000km buffer around point
    geo_data = geo_data.to_crs({'init': 'esri:54032'}) 
    buffer=geo_data.buffer(1000*1000)
    buffer=buffer.to_crs(epsg=4326)

    #create single large polygon from individual buffers
    union_buffer=buffer.unary_union

    #first clip the raster based on this extend 
    raster=rasterio.open(file_dir+'/data/GIS/env_stacked/stacked_env_variables.tif')
    #specify output tif:
    out_tif = file_dir+'/data/GIS/spec_stacked_raster_clip/%s_raster_clip.tif'%spec

    #clip the raster:
    out_img, out_transform = mask(dataset=raster, shapes=[union_buffer],crop=True)

    # Copy the metadata
    out_meta = raster.meta.copy()

    # Parse EPSG code
    epsg_code = int(raster.crs.data['init'][5:])
    out_meta.update({"driver": "GTiff",
                     "height": out_img.shape[1],
                     "width": out_img.shape[2],
                     "transform": out_transform,
                     "crs": pycrs.parse.from_epsg_code(epsg_code).to_proj4()})

    with rasterio.open(out_tif, "w", **out_meta) as dest:
            dest.write(out_img)

Inspect whether clip was correct

In [ ]:
#Inspect the first band of the clipped raster for all species
for key in species_occ_dict:
    ##### Extract occurrence point to plot on the raster (see if correct area was clipped)
    data=species_occ_dict[key]
    spec = key
    data['coordinates'] = list(zip(data["decimal_longitude"], data["decimal_latitude"]))
    data['coordinates'] = data["coordinates"].apply(Point)
    geo_data=geopandas.GeoDataFrame(data, geometry='coordinates',crs={'init' :'epsg:4326'})
    ####open the clipped raster
    clipped = rasterio.open(file_dir+'/data/GIS/spec_stacked_raster_clip/%s_raster_clip.tif'%spec)
    array = clipped.read(1)
    array_data = clipped.read(1,masked=True)
    array_meta = clipped.profile
   
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(array_data,cmap="gist_earth",interpolation="none",vmin=0,
    # Here you must set the spatial extent or else the data will not line up with your geopandas layer
    extent=plotting_extent(clipped),)
    spec_plots_points=geo_data["coordinates"]
    spec_plots_points.plot(ax=ax,
                       marker='o',
                       markersize=20,
                       color='red')
    ax.set_title("%s \n Raster clip and occurrence points"%spec,
             fontsize=20)
    plt.show()
#Works!

#### Part 2
- now that we have the clipped raster we can use it to try and make a random selection of pseudo absence points
- we first open the raster
- then we separate those cells that actually contain pixel values (excluding the sea)
- we calculate the longitude and latitude of the centre point of these cells <br>
  (the environmental variable values do not vary within each cell so it doesn't matter if each points is in the centre)
- we make a random selection of 10.0000 positions
- we add the longitude and latitude values of these to to the dataset and export it  

In [ ]:
stack_path=file_dir+'/data/GIS/env_stacked/stacked_env_variables.tif'
r2=gdal.Open(stack_path)


for key in species_occ_dict: 
    print(key)
    #lon_lat presence points
    presence_data = species_occ_dict[key]
    presence_data["present/pseudo_absent"]=1
    spec = key
    long=presence_data["decimal_longitude"]
    lati=presence_data["decimal_latitude"]
    long=pd.Series.tolist(long)
    lati=pd.Series.tolist(lati)
   
    
    ###############################################################################################################################
    
    ##############
    #sample random locations from specie raster clip excluding sea and presence cells
    ##############
    
    #read raster
    #src=rasterio.open(file_dir+'/data/GIS/spec_stacked_raster_clip/%s_raster_clip.tif'%spec)
    #array=src.read_masks(1)
    
    #set raster cell mask values of presence locations to 1
    #for i in range(0,len(presence_data)):
     #   row,col=src.index(long[i],lati[i])
      #  array[row,col]=1
    
    #fig, ax = plt.subplots(figsize=(10, 10))
    #ax.imshow(array,cmap="gray")
    #ax.set_title("%s"%spec,
     #        fontsize=20)
   # plt.show()
   # print(len(presence_data), "number of presences")
    
    #(y_index, x_index) = np.nonzero(array > 1) #ensures sampling from array with values >1 (excluding presences and no-data/sea)
    
    
   # r = gdal.Open(file_dir+'/data/GIS/spec_stacked_raster_clip/%s_raster_clip.tif'%spec)
   # (upper_left_x, x_size, x_rotation, upper_left_y, y_rotation, y_size) = r.GetGeoTransform()
   # x_coords = x_index * x_size + upper_left_x + (x_size / 2) #add half the cell size
   # y_coords = y_index * y_size + upper_left_y + (y_size / 2) #to centre the point

    #lon_lat_array=np.stack((x_coords,y_coords)).T

    #random_sample_size=0
    #len_p=int(len(presence_data))
    
    #if len_p >1000:
     #   random_sample_size=len_p
    #else: 
     #   random_sample_size=1000
   
    #inner_random_sample_lon_lats=lon_lat_array[np.random.choice(lon_lat_array.shape[0], random_sample_size, replace=False), :] ##
    #print(len(inner_random_sample_lon_lats), "number of inner pseudo absences")

    ############################################################################
    #sample random locations from world raster excluding sea and presence cells
    ############################################################################
    
    src=rasterio.open(stack_path)
    array=src.read_masks(1)
    
    #set raster cell mask values of presence locations to 1
    for i in range(0,len(presence_data)):
        row,col=src.index(long[i],lati[i])
        array[row,col]=1
    
    #fig, ax = plt.subplots(figsize=(10, 10))
    #ax.imshow(array,cmap="gray")
    #ax.set_title("%s"%spec,
     #        fontsize=20)
   # plt.show()
    #print(len(presence_data), "number of presences")
    
    (y_index_2, x_index_2) = np.nonzero(array > 1) #ensures sampling from array with values >1 (excluding presences and no-data/sea)
    
    r=r2
    (upper_left_x, x_size, x_rotation, upper_left_y, y_rotation, y_size) = r.GetGeoTransform()
    x_coords = x_index_2 * x_size + upper_left_x + (x_size / 2) #add half the cell size
    y_coords = y_index_2 * y_size + upper_left_y + (y_size / 2) #to centre the point

    lon_lat_array=np.stack((x_coords,y_coords)).T

    random_sample_size=0
    len_p=int(len(presence_data))
    
    if len_p >1000:
        random_sample_size=len_p
    else: 
        random_sample_size=2000
   
    outer_random_sample_lon_lats=lon_lat_array[np.random.choice(lon_lat_array.shape[0], random_sample_size, replace=False), :] ##
    #print(len(outer_random_sample_lon_lats), "number of outer pseudo absences")
    
     
    ###############################################################################################################################
    
    #Add random points to dataset
    lon=[]
    lat=[]
    psa=[0]*(random_sample_size)
    taxon=["%s"%spec]*(random_sample_size)
    gbif=["no_id"]*(random_sample_size)

   # for item in inner_random_sample_lon_lats:
    #    longitude=item[0]
    #    latitude=item[1]
    #    lon.append(longitude)
    #    lat.append(latitude)

    for item in outer_random_sample_lon_lats:
        longitude=item[0]
        latitude=item[1]
        lon.append(longitude)
        lat.append(latitude)
        
    ###Dataset including n pseudo-absence points for capriolus capriolus
    new_data=pd.DataFrame({"gbif_id": gbif,"taxon_name":taxon,"decimal_longitude": lon, "decimal_latitude":lat, "present/pseudo_absent": psa})
    data=pd.concat([presence_data,new_data],ignore_index=True)
    data=data[['taxon_name','gbif_id','decimal_longitude','decimal_latitude','present/pseudo_absent']]
    data["taxon_name"]=spec
    data["row_n"]=np.arange(len(data))
     
    long=data["decimal_longitude"]
    lati=data["decimal_latitude"]
    long=pd.Series.tolist(long)
    lati=pd.Series.tolist(lati)
    
    #print(len(data),"lenght data with pseudo absences pre-filtering")
    
    #read raster
    src=rasterio.open(stack_path)
    array=src.read_masks(1)
    
    ##remove potential presence locations in the sea##
    #for i in range(1,182):
     #   print(i)
      #  array=src.read_masks(i)
       # for j in range(0,len(data)):
        #    row,col=src.index(long[j],lati[j])
         #   if array[row,col] == 0:
          #      data=data[data.row_n != j]     
    #print(len(data), "length data with pseudo absences post-filtering")
    
    
    data=data.reset_index(drop=True)
    data.to_csv(file_dir + "/data/spec_ppa/%s_ppa_dataframe.csv"%spec)

#### Part 3 
- finally we can extract the environmental variable values underneath the occurrence and pseudo-absence points
- we need to scale these environmental values for later training by taking their mean and std_dev
- below is a code snippet, but because it requires a long time to run in jupyter the process is best split (see extract_env_variables1-4.py files)

In [ ]:
#access file with list of taxa names
taxa=pd.read_csv(file_dir+"/data/SQL_filtered_gbif/taxa_list.txt",header=None)
taxa.columns=["taxon"]

In [ ]:
raster=rasterio.open(file_dir+'/data/GIS/env_stacked/stacked_env_variables.tif')


array = raster.read()
profile=raster.profile

with open(file_dir+'/data/GIS/env_bio_mean_std.txt','w+') as file:
    file.write("band"+"\t"+"mean"+"\t"+"std_dev"+"\n")
    file.close()


    
for i in range(1,186):
    print(i)
    profile.update(count=1)
    band=raster.read(i)
    band[band < -9999] = -9999
    where_are_NaNs = np.isnan(band)
    band[where_are_NaNs] = -9999
    band_masked = np.ma.masked_array(band, mask=(band == -9999))

    mean=band_masked.mean()
    std_dev=np.std(band_masked)

    #mean=np.nanmean(band)
    print(mean)
    #std_dev=np.nanstd(band)
    print(std_dev)

    with open(file_dir+'/data/GIS/env_bio_mean_std.txt','a') as file:
        file.write(str(i)+"\t"+str(mean)+"\t"+str(std_dev)+"\n")
            

In [ ]:
for key in species_occ_dict:    
    #load occurrence data and set initial projection
    data=species_occ_dict[key]
    print("processing "+key)
    spec = key


    data['coordinates'] = list(zip(data["decimal_longitude"], data["decimal_latitude"]))
    data['coordinates'] = data["coordinates"].apply(Point)
    data["present/pseudo_absent"]=1
    geo_data=geopandas.GeoDataFrame(data, geometry='coordinates',crs={'init' :'epsg:4326'})

    #change projection to azimuthal equidistant to calculate 1000km buffer around point
    geo_data = geo_data.to_crs({'init': 'esri:54032'}) 
    buffer=geo_data.buffer(1000*1000)
    buffer=buffer.to_crs(epsg=4326)

    #create single large polygon from individual buffers
    union_buffer=buffer.unary_union

    #first clip the raster based on this extend 
    raster=rasterio.open(file_dir+'/data/GIS/env_stacked/stacked_env_variables.tif')
    #specify output tif:
    out_tif = file_dir+'/data/GIS/spec_stacked_raster_clip/%s_raster_clip.tif'%spec

    #clip the raster:
    out_img, out_transform = mask(dataset=raster, shapes=[union_buffer],crop=True)
   
    # Copy the metadata
    out_meta = raster.meta.copy()

    # Parse EPSG code
    epsg_code = int(raster.crs.data['init'][5:])
    out_meta.update({"driver": "GTiff",
                     "height": out_img.shape[1],
                     "width": out_img.shape[2],
                     "transform": out_transform,
                     "crs": pycrs.parse.from_epsg_code(epsg_code).to_proj4()})

    with rasterio.open(out_tif, "w", **out_meta) as dest:
            dest.write(out_img)

In [ ]:
###column variable names
with open(file_dir+'/data/GIS/env_stacked/variable_list.txt') as f:
      new_cols = f.readlines()

var_names=[]
for item in new_cols:
    item=item.replace("\n","")
    var_names.append(item) 
len(var_names)

In [ ]:
src=rasterio.open(file_dir+'/data/GIS/env_stacked/stacked_env_variables.tif')
inRas=gdal.Open(file_dir+'/data/GIS/env_stacked/stacked_env_variables.tif')

for i in taxa["taxon"][:]:
    data = pd.read_csv(file_dir+"/data/spec_ppa/%s_ppa_dataframe.csv"%i)
    
    spec = data["taxon_name"][0]
    spec = spec.replace(" ","_")
    
    print("processing species ", spec)
    

    #get all col and row values for species locations 
    len_pd=np.arange(len(data))
    long=data["decimal_longitude"]
    lati=data["decimal_latitude"]
    ppa=data["present/pseudo_absent"]

    lon=long.values
    lat=lati.values

    row=[]
    col=[]

    #src=rasterio.open(raster)

    for i in len_pd:
        row_n, col_n = src.index(lon[i], lat[i])# spatial --> image coordinates
        row.append(row_n)
        col.append(col_n)

    ##opening raster as 3d numpy array
    #inRas=gdal.Open(raster)
    myarray=inRas.ReadAsArray()
    #print(myarray.shape)
    #print(type(myarray))

    #collect file with mean and std_dev for each band
    mean_std=pd.read_csv(file_dir+'/data/GIS/env_bio_mean_std.txt',sep="\t")
    mean_std=mean_std.to_numpy()


    ########################################################
    #extract the values for all bands and prepare input data
    ########################################################
    X=[]
    species =["%s"%spec]*int(len(row))

    for j in range(0,186):
        band=myarray[j]
        x=[]

        for i in range(0,len(row)):
            value= band[row[i],col[i]]
            if j < 46:
                if value <-1000:
                    value=np.nan
                else:  
                    value = ((value - mean_std.item((j,1))) / mean_std.item((j,2)))#scale values
                x.append(value)
                
            if j >= 46:
                if value <-1000:
                    value=np.nan
                else:  
                    value=value
                x.append(value)
        X.append(x)
    
    

    #set as numpy 2d array
    X =np.array([np.array(xi) for xi in X])
    #X
    
    #transform into dataframe and include row and column values
    df=pd.DataFrame(X) 
    df=df.T
    #df=
    #df
    
    df["present/pseudo_absent"]=ppa
    df["decimal_latitude"]=lati
    df["decimal_longitude"]=long
    df["taxon_name"]=species
    df["present/pseudo_absent"]=ppa
    df["row_n"]=row
    df.rename(columns=dict(zip(df.columns[0:186], var_names)),inplace=True)
    df=df.dropna(axis=0, how='any')
    #df.head()
    input_data=df
    ##save input dataframe
    input_data.to_csv(file_dir +"/data/spec_ppa_env/%s_env_dataframe.csv"%spec)

In [ ]:
##opening raster as 3d numpy array
inRas=gdal.Open(file_dir+'/data/GIS/env_stacked/stacked_env_variables.tif')
myarray=inRas.ReadAsArray()
print(myarray.shape)
print(type(myarray))

#get all col and row values for all cells on land 
df=pd.read_csv(file_dir+'/data/GIS/world_locations_to_predict.csv')
len_pd=np.arange(len(df))
print(len_pd)
lon=df["decimal_longitude"]
lat=df["decimal_latitude"]
lon=lon.values
lat=lat.values

row=[]
col=[]

src=rasterio.open(file_dir+'/data/GIS/env_stacked/stacked_env_variables.tif')

for i in len_pd:
    row_n, col_n = src.index(lon[i], lat[i])# spatial --> image coordinates
    row.append(row_n)
    col.append(col_n)

#collect file with mean and std_dev for each band
mean_std=pd.read_csv(file_dir+'/data/GIS/env_bio_mean_std.txt',sep="\t")
mean_std=mean_std.to_numpy()


########################################################
#extract the values for all bands and prepare input data
########################################################
X=[]

for j in range(0,186):
    print(j)
    band=myarray[j]
    x=[]

    for i in range(0,len(row)):
        if j < 46:
            value= band[row[i],col[i]]
            value = ((value - mean_std.item((j,1))) / mean_std.item((j,2)))#scale values
            x.append(value)
        if j >= 46:
            value= band[row[i],col[i]]
            x.append(value)
    X.append(x)

#X.append(x)
#include row and column values
X.append(row)
X.append(col)
#set as numpy 2d array
X =np.array([np.array(xi) for xi in X])

df=pd.DataFrame(X)

df=df.T
df.rename(columns=dict(zip(df.columns[0:186], var_names)),inplace=True)
df=df.dropna(axis=0, how='any')
df.head()

input_X=df.iloc[:,0:186]
np.shape(input_X)

row=df[186]
col=df[187]

row_col=pd.DataFrame({"row":row,"col":col})

#convert dataframe back to numpy array
input_X=input_X.values
#convert rows and col indices back to array
row=row.values
col=col.values

#save
prediction_array=np.save(file_dir+'/data/GIS/world_prediction_array.npy',input_X)
prediction_pandas=row_col.to_csv(file_dir+'/data/GIS/world_prediction_row_col.csv')